# 1. Data Scraping

The scrape code was created using the tutorial available on the webpage:
https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas

By: Alan Hylands

In [1]:
import urllib.request

Downloading the page

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

Extracting the date from the page

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup = BeautifulSoup(page, "lxml")

In [5]:
all_tables=soup.find_all("table")

In [6]:
right_table=soup.find('table', class_='wikitable')

In [7]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        if cells[1].find(text=True)[:-1]!="Not assigned":
            #The strings have a '\n' at their end
            # that is why I will remove the lat letter
            A.append(cells[0].find(text=True)[:-1])
            B.append(cells[1].find(text=True)[:-1])
            #Some zip codes have multiple neighbourhoods
            # that is why I will take only the first one of them
            C.append(cells[2].find(text=True)[:-1].partition('/')[0])

Putting the data in a DataFrame

In [74]:
import pandas as pd
df=pd.DataFrame(A,columns=['Zip'])
df['Borough']=B
df['Neighborhood']=C
df

,Zip,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# 2. Downloading the Data of Neighborhoods and Venues

Importing the necessary libraries

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Getting the Coordinates of the different Neighborhoods

In [10]:
AA=[]
geolocator = Nominatim(user_agent="foursquare_agent")
for ii,cc in enumerate(np.array(df)):
    address=cc[2]+', '+cc[1]+', Canada'
    location = geolocator.geocode(address)
    if location: #Some locations have no data
        latitude = location.latitude
        longitude = location.longitude
        #Saving all the data of the locations in an array
        AA.append([cc[0],cc[1],cc[2], latitude, longitude])

In [11]:
print(ii)

102


In [73]:
#Turning the array into a DataFrame
df2=pd.DataFrame(AA)
df2=df2.rename(columns={0: "Zip", 1: "Borough" , 2: "Neighborhood" , 3: "Latitude" , 4: "Longitude"})

df2

,Zip,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M6A,North York,Lawrence Manor,43.722079,-79.437507
3,M7A,Downtown Toronto,Queen's Park,43.663217,-79.386290
4,M9A,Etobicoke,Islington Avenue,43.622575,-79.514215
5,M1B,Scarborough,Malvern,43.809196,-79.221701
6,M3B,North York,Don Mills,43.775347,-79.345944
7,M6B,North York,Glencairn,43.708712,-79.440685
8,M9B,Etobicoke,West Deane Park,43.663199,-79.568568
9,M1C,Scarborough,Rouge Hill,43.780271,-79.130499


### Obtaining the coordinates of Toronto

In [13]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Visualizing the locations of the Neighborhoods on the map of Toronto

In [14]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df2[3], df2[4], df2[2]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Connecting to FourSquare to get the venues

In [15]:
CLIENT_ID = 'BOW3IAUBWDY4NLFLNVA3WVXBXBN3C5VGRMCTTEX40CU4DTN2' # your Foursquare ID
CLIENT_SECRET = 'SB1RETIR5R3CMPX40JPQCA43BZFE4ZJJS5E1PJTEK23KJNR0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BOW3IAUBWDY4NLFLNVA3WVXBXBN3C5VGRMCTTEX40CU4DTN2
CLIENT_SECRET:SB1RETIR5R3CMPX40JPQCA43BZFE4ZJJS5E1PJTEK23KJNR0


### Getting the venues in each of the Neighborhoods withing a 500m radius from the Neighborhood location

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
Toronto_venues = getNearbyVenues(names=df2[2],
                                   latitudes=df2[3],
                                   longitudes=df2[4]
                                  )


Parkwoods
Victoria Village
Lawrence Manor 
Queen's Park 
Islington Avenue
Malvern 
Don Mills
Glencairn
West Deane Park 
Rouge Hill 
Don Mills
Woodbine Heights
Eringate 
Guildwood 
The Beaches
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor 
Thorncliffe Park
Richmond 
Dufferin 
Scarborough Village
Fairview 
Northwood Park 
East Toronto
Little Portugal 
Kennedy Park 
Bayview Village
Downsview
The Danforth West 
Brockton 
Golden Mile 
York Mills 
Downsview
North Park 
Humber Summit
Cliffside 
Willowdale 
Downsview
Studio District
Bedford Park 
Humberlea 
Birch Cliff 
Willowdale
Downsview
Lawrence Park
Roselawn
Runnymede 
Weston
Dorset Park 
York Mills West
Davisville North
Forest Hill North & West
High Park 
Westmount
Wexford 
Willowdale
The Annex 
Parkdale 
Kingsview Village 
Agincourt
Davisville
Runnymede 
Clarks Corners 
Moore Park 
Milliken 
Summerhill West 
New Toronto 
South Steeles 
Steeles West 
Rosedale
Alderwood 
Northwest
Upper Rouge
The Ki

In [19]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2005, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
3,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
4,Parkwoods,43.7588,-79.320197,TD Canada Trust,43.757569,-79.314976,Bank


In [20]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,13,13,13,13,13,13
Alderwood,9,9,9,9,9,9
Bathurst Manor,9,9,9,9,9,9
Bayview Village,12,12,12,12,12,12
Bedford Park,1,1,1,1,1,1
Birch Cliff,5,5,5,5,5,5
Brockton,22,22,22,22,22,22
Cedarbrae,26,26,26,26,26,26
Central Bay Street,49,49,49,49,49,49


In [21]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 244 uniques categories.


## Preparing for the clustering

In [22]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
Toronto_onehot.shape

(2005, 244)

In [27]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Music Store,Music Venue,Nail Salon,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.307692,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,

In [28]:
Toronto_grouped.shape

(75, 244)

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Train Station,Shopping Mall,Cantonese Restaurant,Asian Restaurant,Coffee Shop,Korean Restaurant,Food Court,Hong Kong Restaurant,Vietnamese Restaurant
1,Alderwood,Pizza Place,Gym,Pharmacy,Pool,Coffee Shop,Sandwich Place,Pub,Skating Rink,Eastern European Restaurant,Dive Bar
2,Bathurst Manor,Pizza Place,Bagel Shop,Bakery,Mexican Restaurant,Park,Grocery Store,Empanada Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Bayview Village,Bank,Outdoor Supply Store,Fish Market,Sporting Goods Shop,Metro Station,Gas Station,Sandwich Place,Fast Food Restaurant,Pizza Place,Breakfast Spot
4,Bedford Park,Construction & Landscaping,Women's Store,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


# 3. Clustering the Neighborhoods

In [63]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 4, 3, 2, 2, 2, 2])

In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on=2)
Toronto_merged=Toronto_merged.dropna() #Dropping the NaN rows
Toronto_merged # check the last columns!

,0,1,2,3,4,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.758800,-79.320197,2.0,Convenience Store,Coffee Shop,Shopping Mall,Liquor Store,Laundry Service,Gas Station,Caribbean Restaurant,Chinese Restaurant,Discount Store,Bank
1,M4A,North York,Victoria Village,43.732658,-79.311189,2.0,Thai Restaurant,Middle Eastern Restaurant,Bus Line,Mediterranean Restaurant,Women's Store,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,M6A,North York,Lawrence Manor,43.722079,-79.437507,2.0,Kids Store,Doctor's Office,Park,Electronics Store,Bank,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,M7A,Downtown Toronto,Queen's Park,43.663217,-79.386290,2.0,Burger Joint,Park,Japanese Restaurant,Diner,Coffee Shop,Italian Restaurant,Gay Bar,Sake Bar,Burrito Place,Falafel Restaurant
4,M9A,Etobicoke,Islington Avenue,43.622575,-79.514215,2.0,Restaurant,Coffee Shop,Italian Restaurant,BBQ Joint,Liquor Store,Bar,Sushi Restaurant,Furniture / Home Store,Sporting Goods Shop,Yoga Studio
5,M1B,Scarborough,Malvern,43.809196,-79.221701,2.0,Pizza Place,Pharmacy,Fast Food Restaurant,Gym / Fitness Center,Skating Rink,Park,Grocery Store,Bubble Tea Shop,Sandwich Place,Falafel Restaurant
6,M3B,North York,Don Mills,43.775347,-79.345944,2.0,Clothing Store,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Restaurant,Tea Room,Food Court,Bakery,Bank,Greek Restaurant
7,M6B,North York,Glencairn,43.708712,-79.440685,2.0,Playground,Asian Restaurant,Sushi Restaurant,Grocery Store,Japanese Restaurant,Pub,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
9,M1C,Scarborough,Rouge Hill,43.780271,-79.130499,2.0,Bus Line,Train Station,Women's Store,Doctor's Office,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
10,M3C,North York,Don Mills,43.775347,-79.345944,2.0,Clothing Store,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Restaurant,Tea Room,Food Court,Bakery,Bank,Greek Restaurant


In [65]:
#Converting the cluster numbering into integers
Toronto_merged = Toronto_merged.astype({'Cluster Labels': int})


In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged[3], Toronto_merged[4], Toronto_merged[2], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters